In [2]:
import xml.etree.ElementTree
import os,glob,sys
import pandas as pd
from sklearn.decomposition import PCA
import pylab as plt
import seaborn as sns
import numpy as np

%load_ext autoreload
%autoreload 2

sys.path.append("..")
from mzmine import pick_peaks, align, match_aligned_to_original, load_aligned_peaks


### Load text files with aligned data


In [3]:
with_mod = pd.read_csv('/Users/anamaria/Desktop/mzmine/221119/topfeatures50_filled.txt', sep = '\t')
without_mod = pd.read_csv('/Users/anamaria/Desktop/mzmine/210120/topfeatures50_filled.txt', sep = '\t')

In [28]:
nomod = {}
mod = {}
for _,row in without_mod.iterrows():
    mz = row["row m/z"]
    rt = row["row retention time"]
    idc = np.int(row["custom_id"])
    nomod[round(mz,2)] = idc
for _,row in with_mod.iterrows():
    mz = row["row m/z"]
    rt = row["row retention time"]
    idc = np.int(row["custom_id"])
    mod[round(mz,2)] = idc

In [37]:
def intersection(lst1, lst2): 
    return set(lst1).intersection(lst2) 
      
inters  = intersection(mod.keys(),  nomod.keys())

In [55]:
found = []
notfound = []
for i in mod:
    if i in list(inters):
        found = mod[i]
    else:
        notfound = mod[i]


In [ ]:
mod_sig = with_mod[with_mod['adj p-val'] < 0.05]
nomod_sig = without_mod[without_mod['adj p-val'] < 0.05]

mal = pd.read_csv('/Users/anamaria/Documents/pymz/mzmine/221119/mal/topfeatures_filled.txt', sep = '\t')
mal_sig = mal[mal['adj-p'] < 0.05]

vl = pd.read_csv('/Users/anamaria/Documents/pymz/mzmine/221119/vl/topfeatures_filled.txt', sep = '\t')
vl_sig = vl[vl['adj-p'] < 0.05]
zik = pd.read_csv('/Users/anamaria/Documents/pymz/mzmine/221119/zik/topfeatures_filled.txt', sep = '\t')
zik_sig = zik[zik['adj-p'] < 0.05]

mal_new = pd.read_csv('/Users/anamaria/Documents/pymz/mzmine/221119/mal/topfeatures_filled_changed_rt.txt', sep = '\t')
mal_sig_new = mal_new[mal_new['adj-p'] < 0.05]

vl_new = pd.read_csv('/Users/anamaria/Documents/pymz/mzmine/221119/vl/topfeatures_filled_changed_rt.txt', sep = '\t')
vl_sig_new = vl_new[vl_new['adj-p'] < 0.05]


nomod = {}
mod = {}

mod_rt = []
nomod_rt = []

mod_mz = []
nomod_mz = []

for _,row in nomod_sig.iterrows():
    mz = row["row m/z"]
    rt = row["row retention time"]
    idc = np.int(row["custom_id"])
    nomod[round(mz,2)] = idc
    nomod_rt.append(rt)
    nomod_mz.append(mz)
    
for _,row in mod_sig.iterrows():
    mz = row["row m/z"]
    rt = row["row retention time"]
    idc = np.int(row["custom_id"])
    mod[round(mz,2)] = idc
    mod_rt.append(rt)
    mod_mz.append(mz)

def intersection(lst1, lst2): 
    return set(lst1).intersection(lst2) 
      
inters  = intersection(mod.keys(),  nomod.keys())


found = []
notfound = []
for i in mod:
    if i in list(inters):
        found.append(mod[i])
    else:
        notfound.append(mod[i])

        
names_notfound = []
index = 0
for i in notfound:
    name  = str(data.loc[i][5]).strip()
    if name != "NaN":
        if name != "nan":
            index+=1
            print("========",index,"=======")
            print(name)
            names_notfound.append(name)
            print("")   
            
            
names_found = []
index = 0
for i in found:
    name  = str(data.loc[i][5]).strip()
    if name != "NaN":
        if name != "nan":
            index+=1
            print("========",index,"=======")
            print(name)
            names_found.append(name)
            print("")
            
for i in (list(intersection(names_notfound, names_found))):
    print("=============")
    print(i)
    
for i in names_notfound:
    if i not in (list(intersection(names_notfound, names_found))):
        print("=============")
        print(i)

### Check intersection metabolites by m/z

In [ ]:
inters = 0
for _,row in nomod_sig.iterrows():
    mz = row['row m/z']
    rt = row['row retention time']
    for _,rowc in mod_sig.iterrows():
        mzc = rowc['row m/z']
        rtc = rowc['row retention time']
        if (mz < (mzc+0.001)  or  mz > (mzc-0.001)):
            inters+=1
            break
            #print(row)

In [ ]:
print("Mal sig: {}, VL sig: {}, Zik sig: {}".format(len(mal_sig), len(vl_sig), len(zik_sig)))

In [ ]:
c=0
z=0
for _,row in nomod_sig.iterrows():
    mz = row['row m/z']
    rt = row['row retention time']
    for _,row in vl_sig.iterrows():
        mz_vl = row['row m/z']
        rt_vl = row['row retention time']
        if mz_vl > (mz - 0.01) and mz_vl < (mz + 0.01):
            c+=1
            if rt_vl > (rt - 300) and rt_vl < (rt +300):
                z+=1
print(c,z)

### Check retention times

In [ ]:
plt.hist(mod_rt)
plt.hist(nomod_rt, alpha = 0.5)
plt.xlabel('RT')
plt.ylabel('Peaks (#)')


unpreprocessed data: Most of the peaks which align are found in the interval [3,7] and very few in [8,14]



preprocessed data: More peaks align in the interval [8,14]

So the modelling mainly influences the metabolites eluted after 8 min.

In [ ]:
c=0
z=0
for _,row in vl_sig_new.iterrows():
    mz = row['row m/z']
    mz = np.round(mz,3)
    rt = row['row retention time']
    for _,rowl in zik_sig.iterrows():
        mz_vl = rowl['row m/z']
        
        rt_vl = rowl['row retention time']
        if mz_vl > (mz - 0.01) and mz_vl < (mz + 0.01):
            c+=1
            if rt_vl > (rt - 30/60) and rt_vl < (rt +30/60):
                z+=1
                
print(c,z)


c=0
z=0
for _,row in mal_sig.iterrows():
    mz = row['row m/z']
    rt = row['row retention time']
    for _,row in zik_sig.iterrows():
        mz_vl = row['row m/z']
        rt_vl = row['row retention time']
        if mz_vl > (mz - 0.01) and mz_vl < (mz + 0.01):
            c+=1
            if rt_vl > (rt - 300) and rt_vl < (rt +300):
                z+=1
print(c,z)

cv=0
zv=0
for _,row in mod_sig.iterrows():
    mz = row['row m/z']
    rt = row['row retention time']
    for _,rowl in nomod_sig.iterrows():
        mz_vl = rowl['row m/z']
        rt_vl = rowl['row retention time']
        if mz_vl > (mz - 0.01) and mz_vl < (mz + 0.01):
            cv+=1
            if rt_vl > (rt - 300/60) and rt_vl < (rt +300/60):
                zv+=1
                print(mz, rt)
                print(mz_vl, rt_vl)
            print('-------')
print(cv,zv)

plt.hist(mod_mz,)
plt.hist(nomod_mz, alpha = 0.5)

plt.xlabel('m/z')
plt.ylabel('Peaks (#)')

plt.hist(mod_mz, color = 'pink')
plt.hist(nomod_mz, alpha = 0.5)
plt.hist(mod_sig['row m/z'], alpha = 0.4)
plt.hist(nomod_sig['row m/z'], alpha = 0.3)


plt.xlabel('m/z')
plt.ylabel('Peaks (#)')

In [ ]:
plt.figure(figsize=(15,5))
plt.hist(mod_sig['row m/z'], bins=100)
plt.hist(nomod_sig['row m/z'], alpha = 0.5, bins=100)

plt.xlabel('m/z')
plt.ylabel('Peaks (#)')

sns.distplot(mod_sig['row m/z'])
sns.distplot(nomod_sig['row m/z'])

sns.distplot(mod_sig['row retention time'])
sns.distplot(nomod_sig['row retention time'])

plt.figure(figsize=(15,5))
plt.hist(mod_sig['row retention time'], bins=100)
plt.hist(nomod_sig['row retention time'], alpha = 0.5, bins=100)

plt.xlabel('RT')
plt.ylabel('Peaks (#)')


plt.figure(figsize=(15,5))
plt.hist(mod_sig['row m/z'], bins=50)
plt.hist(nomod_sig['row m/z'], alpha = 0.5, bins=50)
plt.hist(vl_sig['row m/z'], alpha = 0.5, bins=50)
plt.xlabel('m/z')
plt.ylabel('Peaks (#)')

plt.figure(figsize=(15,5))
plt.hist(mod_sig['row m/z'], bins=50)
plt.hist(nomod_sig['row m/z'], alpha = 0.5, bins=50)
plt.hist(zik_sig['row m/z'], alpha = 0.8, bins=50)
plt.xlabel('m/z')
plt.ylabel('Peaks (#)')

plt.figure(figsize=(15,5))
plt.hist(mod_sig['row m/z'], bins=50)
plt.hist(nomod_sig['row m/z'], alpha = 0.5, bins=50)
plt.hist(mal_sig['row m/z'], alpha = 0.8, bins=50)
plt.xlabel('m/z')
plt.ylabel('Peaks (#)')


plt.hist(mod_rt)
plt.hist(nomod_rt, alpha = 0.5)
plt.hist(zik['row retention time'], alpha = 0.5)

plt.xlabel('RT')
plt.ylabel('Peaks (#)')

plt.hist(mod_rt)
plt.hist(nomod_rt, alpha = 0.5)
plt.hist(mal['row retention time'], alpha = 0.5)
plt.xlabel('RT')
plt.ylabel('Peaks (#)')